In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import mediapipe as mp

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import math
from happytransformer import HappyTextToText
from happytransformer import TTSettings

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence = 0.8, min_tracking_confidence = 0.5)

In [3]:
actions_single_flexible = np.array(['--', 'L', 'name', 'this', 'jayga'])
actions_single_fixed = np.array(['I_word', 'hello', 'my', 'india(n)'])
actions_dual_flexible = np.array(['--','A', 'B', "I_lett", 'J', 'N', 'W', "language", "sign", "teacher(s)"])
actions_dual_fixed = np.array(['Kol', 'live'])
no_sequences = 120
no_frames = 30

In [4]:
def mediapipe_detection(image, holistic):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = holistic.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, result

In [5]:
def render_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

In [6]:
def get_true_coords(results):
    l_coords = np.zeros(shape = (21, 3), dtype = float)
    r_coords = np.zeros(shape = (21, 3), dtype = float)
    
#     l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
#     r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    if results.left_hand_landmarks:
        for idxL, lh in enumerate(results.left_hand_landmarks.landmark):
            l_coords[idxL] = np.array([lh.x, lh.y, lh.z])
#         l_shift_coords = coord_shift(l_coords, 0)
    
    if results.right_hand_landmarks:
        for idxR, rh in enumerate(results.right_hand_landmarks.landmark):
            r_coords[idxR] = np.array([rh.x, rh.y, rh.z])
#         r_shift_coords = coord_shift(r_coords, 1)
    
    return l_coords, r_coords

In [7]:
def get_shifted_coords(coords_array, num):
    
    coords_zero = np.all((coords_array == 0))
    shifted_coords_array = np.zeros(shape = (21, 3), dtype = float)
    
    new_l_x = 8.24e-01
    new_l_y = 6.7e-01
    new_l_z = 2.25e-07
    
    new_r_x = 1.39e-01
    new_r_y = 6.7e-01
    new_r_z = 2.25e-07
    
    if num == 0:
        shifted_coords_array[0] = np.array([new_l_x, new_l_y, new_l_z])
        shift_factor_x = new_l_x - coords_array[0][0]
        shift_factor_y = new_l_y - coords_array[0][1]
        shift_factor_z = new_l_z - coords_array[0][2]
        for i in range(1, len(coords_array)):
            shifted_coords_array[i] = np.array([(coords_array[i][0] + shift_factor_x), (coords_array[i][1] + shift_factor_y), (coords_array[i][2] + shift_factor_z)])

    else:
        shifted_coords_array[0] = np.array([new_r_x, new_r_y, new_r_z])
        shift_factor_x = new_r_x - coords_array[0][0]
        shift_factor_y = new_r_y - coords_array[0][1]
        shift_factor_z = new_r_z - coords_array[0][2]
        for j in range(1, len(coords_array)):
            shifted_coords_array[j] = np.array([(coords_array[j][0] + shift_factor_x), (coords_array[j][1] + shift_factor_y), (coords_array[j][2] + shift_factor_z)])

    #print(shifted_coords_array)
    return shifted_coords_array

In [8]:
def hand_wrist_distance(coords_array):
    
    wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    for num in range(len(coords_array)):
        wrist_distance[num] = (((coords_array[0][0] - coords_array[num][0])**2) + ((coords_array[0][1] - coords_array[num][1])**2) + ((coords_array[0][2] - coords_array[num][2])**2))**0.5
        
    return wrist_distance

In [9]:
def left_right_hand_distance(l_true_coords, r_true_coords):
    l_r_hand_distance = np.zeros(shape = (21, 1), dtype = float)
    
    for num in range(len(l_true_coords)):
        l_r_hand_distance[num] = (((l_true_coords[num][0] - r_true_coords[num][0])**2) + ((l_true_coords[num][1] - r_true_coords[num][1])**2))**0.5
    
    return l_r_hand_distance

In [10]:
def calculate_angles(results):
    
    #right_elbow, right_shoulder, left_shoulder, left_elbow
    four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    if results.pose_landmarks:
        # angle_joints = [right_elbow, right_shoulder, left_shoulder, left_elbow]
        angle_joints = [[16, 14, 12], [11, 12, 14], [13, 11, 12], [11, 13, 15]]
        
        for joint_index in range(len(angle_joints)):
            
            p1 = np.array([results.pose_landmarks.landmark[angle_joints[joint_index][0]].x, results.pose_landmarks.landmark[angle_joints[joint_index][0]].y])
            p2 = np.array([results.pose_landmarks.landmark[angle_joints[joint_index][1]].x, results.pose_landmarks.landmark[angle_joints[joint_index][1]].y])
            p3 = np.array([results.pose_landmarks.landmark[angle_joints[joint_index][2]].x, results.pose_landmarks.landmark[angle_joints[joint_index][2]].y])
            
            angle = math.degrees(math.atan2(p3[1] - p2[1], p3[0] - p2[0]) - math.atan2(p1[1] - p2[1], p1[0] - p2[0]))
            
            if angle < 0:
                angle = angle + 360
            
            rad_angle = angle * (np.pi / 180)
            
            normalised_angle = (angle / 360)
                
            four_joint_angles[joint_index] = angle
    
    return four_joint_angles

In [11]:
def nose_to_joint(results):
    
    nose_to_joints_distance = np.zeros(shape = (6, 1), dtype = float)
    
    if results.pose_landmarks:
        
        #right_wrist, right_elbow, right_shoulder, left_shoulder, left_elbow, left_wrist
        nose_to_joints = [16, 14, 12, 11, 13, 15]
        
        for distance_idx in range(len(nose_to_joints)):
            
            nose_to_joints_distance[distance_idx] = (((results.pose_landmarks.landmark[0].x - results.pose_landmarks.landmark[nose_to_joints[distance_idx]].x) ** 2) + ((results.pose_landmarks.landmark[0].y - results.pose_landmarks.landmark[nose_to_joints[distance_idx]].y) ** 2)) ** 0.5
    
    return nose_to_joints_distance

In [12]:
def extract_single_flexible_feature_points(results):
    
    l_true_coords = np.zeros(shape = (21, 3), dtype = float)
    r_true_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    r_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    if results:
        l_true_coords, r_true_coords = get_true_coords(results)
        
        if results.left_hand_landmarks:
            l_shift_coords = get_shifted_coords(l_true_coords, 0)
            l_h_wrist_distance = hand_wrist_distance(l_shift_coords)
        
        if results.right_hand_landmarks:
            r_shift_coords = get_shifted_coords(r_true_coords, 1)
            r_h_wrist_distance = hand_wrist_distance(r_shift_coords)
            
    return np.concatenate((l_shift_coords.flatten(), r_shift_coords.flatten(), l_h_wrist_distance.flatten(), r_h_wrist_distance.flatten())).flatten()
#     return np.column_stack((l_shift_coords, r_shift_coords, l_h_wrist_distance, r_h_wrist_distance)).flatten()

In [13]:
def extract_single_fixed_feature_points(results):
    
    l_true_coords = np.zeros(shape = (21, 3), dtype = float)
    r_true_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    r_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    nose_joint_distances = np.zeros(shape = (6, 1), dtype = float)
    
    final_nose_joint_distances = np.zeros(shape = (6, 1), dtype = float)
    
    four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    final_four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    if results:
        l_true_coords, r_true_coords = get_true_coords(results)
        
        four_joint_angles = calculate_angles(results)
        
        nose_joint_distances = nose_to_joint(results)
        
        if results.left_hand_landmarks:
            l_shift_coords = get_shifted_coords(l_true_coords, 0)
            l_h_wrist_distance = hand_wrist_distance(l_shift_coords)
            four_joint_angles[0] = 0
            four_joint_angles[1] = 0
            nose_joint_distances[0] = 0
            nose_joint_distances[1] = 0
            nose_joint_distances[2] = 0
#             final_nose_joint_distances = np.array([0, 0, 0, nose_joint_distances[3], nose_joint_distances[4], nose_joint_distances[5]])
#             final_four_joint_angles = np.array([0, 0, four_joint_angles[2], four_joint_angles[3]], dtype = float)
        
        if results.right_hand_landmarks:
            r_shift_coords = get_shifted_coords(r_true_coords, 1)
            r_h_wrist_distance = hand_wrist_distance(r_shift_coords)
            four_joint_angles[2] = 0
            four_joint_angles[3] = 0
            nose_joint_distances[3] = 0
            nose_joint_distances[4] = 0
            nose_joint_distances[5] = 0
#             final_nose_joint_distances = np.array([nose_joint_distances[0], nose_joint_distances[1], nose_joint_distances[2], 0, 0, 0])
#             final_four_joint_angles = np.array([four_joint_angles[0], four_joint_angles[1], 0, 0], dtype = float)
            
        return np.concatenate((l_shift_coords.flatten(), r_shift_coords.flatten(), l_h_wrist_distance.flatten(), r_h_wrist_distance.flatten(), four_joint_angles.flatten(), nose_joint_distances.flatten())).flatten()
#     return np.column_stack((l_shift_coords, r_shift_coords, l_h_wrist_distance, r_h_wrist_distance, l_r_hand_distance)).flatten()

In [14]:
def extract_dual_flexible_feature_points(results):
    
    l_true_coords = np.zeros(shape = (21, 3), dtype = float)
    r_true_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    r_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    l_r_hand_distance = np.zeros(shape = (21, 1), dtype = float)
    
    if results:
        l_true_coords, r_true_coords = get_true_coords(results)
        
        if results.left_hand_landmarks:
            l_shift_coords = get_shifted_coords(l_true_coords, 0)
            l_h_wrist_distance = hand_wrist_distance(l_shift_coords)
        
        if results.right_hand_landmarks:
            r_shift_coords = get_shifted_coords(r_true_coords, 1)
            r_h_wrist_distance = hand_wrist_distance(r_shift_coords)
            
        if results.left_hand_landmarks and results.right_hand_landmarks:
            l_r_hand_distance = left_right_hand_distance(l_true_coords, r_true_coords)
    
    return np.concatenate((l_shift_coords.flatten(), r_shift_coords.flatten(), l_h_wrist_distance.flatten(), r_h_wrist_distance.flatten(), l_r_hand_distance.flatten())).flatten()
#     return np.column_stack((l_shift_coords, r_shift_coords, l_h_wrist_distance, r_h_wrist_distance, l_r_hand_distance)).flatten()

In [15]:
def extract_dual_fixed_feature_points(results):
    
    l_true_coords = np.zeros(shape = (21, 3), dtype = float)
    r_true_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    r_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    l_r_hand_distance = np.zeros(shape = (21, 1), dtype = float)
    
    nose_joint_distances = np.zeros(shape = (6, 1), dtype = float)
    
    four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    if results:
        l_true_coords, r_true_coords = get_true_coords(results)
        
        four_joint_angles = calculate_angles(results)
        
        nose_joint_distances = nose_to_joint(results)
        
        if results.left_hand_landmarks:
            l_shift_coords = get_shifted_coords(l_true_coords, 0)
            l_h_wrist_distance = hand_wrist_distance(l_shift_coords)
        
        if results.right_hand_landmarks:
            r_shift_coords = get_shifted_coords(r_true_coords, 1)
            r_h_wrist_distance = hand_wrist_distance(r_shift_coords)
            
        if results.left_hand_landmarks and results.right_hand_landmarks:
            l_r_hand_distance = left_right_hand_distance(l_true_coords, r_true_coords)
    
        return np.concatenate((l_shift_coords.flatten(), r_shift_coords.flatten(), l_h_wrist_distance.flatten(), r_h_wrist_distance.flatten(), l_r_hand_distance.flatten(), four_joint_angles.flatten(), nose_joint_distances.flatten())).flatten()
#     return np.column_stack((l_shift_coords, r_shift_coords, l_h_wrist_distance, r_h_wrist_distance, l_r_hand_distance)).flatten()

In [16]:
model_single_flexible = Sequential()
model_single_flexible.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30, 168)))
model_single_flexible.add(LSTM(128, return_sequences = True, activation = 'relu'))
model_single_flexible.add(LSTM(64, return_sequences = False, activation = 'relu'))
model_single_flexible.add(Dense(64, activation = 'relu'))
model_single_flexible.add(Dense(32, activation = 'relu'))
model_single_flexible.add(Dense(actions_single_flexible.shape[0], activation = 'softmax'))
model_single_flexible.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [17]:
model_single_fixed = Sequential()
model_single_fixed.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30, 178)))
model_single_fixed.add(LSTM(128, return_sequences = True, activation = 'relu'))
model_single_fixed.add(LSTM(64, return_sequences = False, activation = 'relu'))
model_single_fixed.add(Dense(64, activation = 'relu'))
model_single_fixed.add(Dense(32, activation = 'relu'))
model_single_fixed.add(Dense(actions_single_fixed.shape[0], activation = 'softmax'))
model_single_fixed.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [18]:
model_dual_flexible = Sequential()
model_dual_flexible.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30, 189)))
model_dual_flexible.add(LSTM(128, return_sequences = True, activation = 'relu'))
model_dual_flexible.add(LSTM(64, return_sequences = False, activation = 'relu'))
model_dual_flexible.add(Dense(64, activation = 'relu'))
model_dual_flexible.add(Dense(32, activation = 'relu'))
model_dual_flexible.add(Dense(actions_dual_flexible.shape[0], activation = 'softmax'))
model_dual_flexible.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [19]:
model_dual_fixed = Sequential()
model_dual_fixed.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30, 199)))
model_dual_fixed.add(LSTM(128, return_sequences = True, activation = 'relu'))
model_dual_fixed.add(LSTM(64, return_sequences = False, activation = 'relu'))
model_dual_fixed.add(Dense(64, activation = 'relu'))
model_dual_fixed.add(Dense(32, activation = 'relu'))
model_dual_fixed.add(Dense(actions_dual_fixed.shape[0], activation = 'softmax'))
model_dual_fixed.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [20]:
model_single_flexible.load_weights('SingleFlexible.h5')

In [21]:
model_single_fixed.load_weights('SingleFixed.h5')

In [22]:
model_dual_flexible.load_weights('DualFlexible.h5')

In [23]:
model_dual_fixed.load_weights('DualFixed.h5')

In [24]:
def generate_sentence(exp_array, final_pred_exp):
    
    sentence = ""

#     ['I']
#     ['I', 'live']
#     ['I', 'live', 'W']
#     ['I', 'live', 'W', 'B']
#     ['I', 'live', 'W', 'B', 'jayga']
    
    mix_word_dict = {"Kolkata" : ['jayga', 'Kol'], "West Bengal" : ['jayga', 'B', 'W']}

    exp_array.append(final_pred_exp)
    
    same_flag = 0
    
#     exp_array = ['I', 'live', 'W', 'B', 'jayga']
#     final_pred_exp = 'jayga'
    
    for key, value in mix_word_dict.items(): 
#         for exp_ind in range(len(exp_array), -1, -1):
        if final_pred_exp == value[0]:
            exp_ind = len(exp_array) - 2
            for v_ind in range(1, len(value)):
                if value[v_ind] == exp_array[exp_ind]:
                    same_flag = 1
                else:
                    same_flag = 0
                    break
                exp_ind -= 1
            if same_flag == 1:
                exp_array[exp_ind + 1] = key
                del exp_array[(exp_ind + 2) : (len(exp_array))]
        else:
            break
            
    for exp in exp_array:      
        if exp == 'A' or exp == 'B' or exp == 'J' or exp == 'N' or exp == 'L' or exp == 'W' or exp == 'I_lett':
            if exp == 'I_lett':
                sentence += "I"
            else:
                sentence += (exp)
        else:
            if exp == 'I_word':
                sentence += ("I" + " ")
            else:
                sentence += (exp + " ")        
#     print(sentence)
                
    return sentence

In [25]:
# gram_model = HappyTextToText("T5",  "prithivida/grammar_error_correcter_v1")

In [26]:
# gram_model.save("Grammar_Correction_Model/")

In [27]:
gram_model_load = HappyTextToText(load_path = "Grammar_Correction_Model/")

06/14/2022 12:59:54 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [28]:
settings = TTSettings(do_sample=True, top_k=10, temperature=0.5,  min_length=1, max_length=100)

In [31]:
sequence_single = []
sequence_dual = []
exp_array = []
sentence = ""
final_correct_sentence = ""
old_pred_char = ''
new_pred_char = ''
display_pred_char = ''
prev_final_pred_char = ''
final_pred_char = ''
count = 0
texty0 = 20
textdiffy = 25
no_hand_count = 0
# threshold = 0.9

# k = 0

imgno = 0

predict_res = []

single_hand_pred_model = model_single_flexible
dual_hand_pred_model = model_dual_flexible

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    image, results = mediapipe_detection(frame, holistic)
    render_landmarks(image, results)
    
    backBlackImage = np.zeros([300, 700, 3])

    backBlackImage.fill(0)
    
    if (results.left_hand_landmarks and not results.right_hand_landmarks) or (results.right_hand_landmarks and not results.left_hand_landmarks):
        
        no_hand_count = 0
        single_flexible_keypoints = extract_single_flexible_feature_points(results)
        single_fixed_keypoints = extract_single_fixed_feature_points(results)
        
        if single_hand_pred_model == model_single_flexible:
            sequence_single.append(single_flexible_keypoints)
            sequence_single = sequence_single[-30:]
        elif single_hand_pred_model == model_single_fixed:
            sequence_single.append(single_fixed_keypoints)
            sequence_single = sequence_single[-30:]
        
        if len(sequence_single) == 30:
            old_pred_char = new_pred_char
            predict_res = single_hand_pred_model.predict(np.expand_dims(sequence_single, axis = 0))[0]
            
            if single_hand_pred_model == model_single_flexible:
                new_pred_char = actions_single_flexible[np.argmax(predict_res)]
            elif single_hand_pred_model == model_single_fixed:
                new_pred_char = actions_single_fixed[np.argmax(predict_res)]
#             print(new_pred_char)
            if new_pred_char != '-':
                if old_pred_char == new_pred_char:
                    count += 1
                else:
                    count = 0
        
#         if single_hand_pred_model == model_single_flexible:
#             count_limit = 20
#         elif single_hand_pred_model == model_single_fixed:
#             count_limit = 20
            
        if count >= 20:
            count = 0
            if new_pred_char == "--":
                single_hand_pred_model = model_single_fixed
                final_pred_char = new_pred_char
            else:
                single_hand_pred_model = model_single_flexible
                final_pred_char = new_pred_char
            sequence_single = []
            
    elif results.left_hand_landmarks and results.right_hand_landmarks:
        
        no_hand_count = 0
        dual_flexible_keypoints = extract_dual_flexible_feature_points(results)
        dual_fixed_keypoints = extract_dual_fixed_feature_points(results)
        
        if dual_hand_pred_model == model_dual_flexible:
            sequence_dual.append(dual_flexible_keypoints)
            sequence_dual = sequence_dual[-30:]
        elif dual_hand_pred_model == model_dual_fixed:
            sequence_dual.append(dual_fixed_keypoints)
            sequence_dual = sequence_dual[-30:]
        
        if len(sequence_dual) == 30:
            old_pred_char = new_pred_char
            predict_res = dual_hand_pred_model.predict(np.expand_dims(sequence_dual, axis = 0))[0]
            
            if dual_hand_pred_model == model_dual_flexible:
                new_pred_char = actions_dual_flexible[np.argmax(predict_res)]
            elif dual_hand_pred_model == model_dual_fixed:
                new_pred_char = actions_dual_fixed[np.argmax(predict_res)]
#             print(new_pred_char)
            if new_pred_char != '-':
                if old_pred_char == new_pred_char:
                    count += 1
                else:
                    count = 0
        
        if count >= 15:
            count = 0
            if new_pred_char == "--":
                dual_hand_pred_model = model_dual_fixed
                final_pred_char = new_pred_char
            else:
                dual_hand_pred_model = model_dual_flexible
                final_pred_char = new_pred_char
            sequence_dual = []
    else:
        new_pred_char = '-'
        final_pred_char = new_pred_char
        no_hand_count += 1
    
    if no_hand_count < 20:
        if final_pred_char != "--" and final_pred_char != '-':

            if prev_final_pred_char != final_pred_char:

                sentence = generate_sentence(exp_array, final_pred_char)
#                 print("Sentence: ", sentence)
            prev_final_pred_char = final_pred_char
            
    elif no_hand_count >= 20:
        if sentence:
            sentence += ". "
#             print(sentence)
            result_sentence = gram_model_load.generate_text(sentence, args=settings)
#             print(result_sentence)
            final_correct_sentence += result_sentence.text
            final_correct_sentence += '\n'
            print("Final Correct Sentence: ", final_correct_sentence)
        exp_array = []
        sentence = ""
        no_hand_count = 20
    
    flipImg = cv2.flip(image, 1)
    cv2.rectangle(flipImg, (0,0), (180,50), (245, 117, 16), -1)
    cv2.putText(flipImg, final_pred_char, (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.rectangle(flipImg, (0, 440), (640,480), (245, 117, 16), -1)
    cv2.putText(flipImg, sentence, (10,466), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    for ind, line in enumerate(final_correct_sentence.split('\n')):
        texty = texty0 + ind * textdiffy
        cv2.putText(backBlackImage, line, (10, texty), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA, False)
    
    
    cv2.imshow("Live Feed", flipImg)
    
#     cv2.imwrite(os.path.join(str(imgno) + '.jpg'), flipImg)
#     imgno += 1
    cv2.imshow("Result Text", backBlackImage)

    if cv2.waitKey(10) & 0Xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Final Correct Sentence:  Hello teacher(s) .

Final Correct Sentence:  Hello teacher(s) .
I live West Bengal.

Final Correct Sentence:  Hello teacher(s) .
I live West Bengal.
this india(n) sign language .

